In [2]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Define constants
data_dir = "Dataset/VISA_Face/VISA_Face"
input_shape = (128, 128, 3)  # Input image dimensions
num_classes = 100
batch_size = 32
epochs = 10
embedding_dim = 64  # Dimensionality of the embedding space

# Function to load and preprocess images


def preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=input_shape[:2])
        img = img_to_array(img) / 255.0  # Normalize pixel values
        return img
    except Exception as e:
        print(f"Error loading image: {image_path}. {e}")
        return None


# Function to recursively search for image files in directories


def find_images(directory):
    images = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                images.append(os.path.join(root, file))
    return images


# Load and preprocess dataset
class_dirs = os.listdir(data_dir)
class_images = {class_dir: find_images(os.path.join(
    data_dir, class_dir)) for class_dir in class_dirs}

pairs = []
labels = []

for class_dir, images in class_images.items():
    for i in range(len(images)):
        for j in range(i + 1, len(images)):
            pairs.append((images[i], images[j]))
            labels.append(1 if class_dir == class_images else 0)

# Split dataset into train and test sets
pairs_train, pairs_test, labels_train, labels_test = train_test_split(
    pairs, labels, test_size=0.2, random_state=42)

# Siamese network architecture


def create_siamese_network(input_shape, embedding_dim):
    input = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    output = layers.Dense(embedding_dim)(x)
    return models.Model(input, output)


# Create Siamese network branches
base_network = create_siamese_network(input_shape, embedding_dim)
input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Calculate L1 distance between the embeddings
distance = tf.abs(processed_a - processed_b)

# Output layer
output = layers.Dense(1, activation='sigmoid')(distance)

# Create Siamese network model
model = models.Model([input_a, input_b], output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# Function to generate batches of pairs


def generate_batch(pairs, labels, batch_size):
    while True:
        batch_indices = random.sample(range(len(pairs)), batch_size)
        batch_pairs = [pairs[i] for i in batch_indices]
        batch_labels = [labels[i] for i in batch_indices]

        X1 = np.zeros((batch_size, *input_shape))
        X2 = np.zeros((batch_size, *input_shape))
        Y = np.array(batch_labels)

        for i, (pair, label) in enumerate(zip(batch_pairs, batch_labels)):
            img1 = preprocess_image(pair[0])
            img2 = preprocess_image(pair[1])
            X1[i] = img1
            X2[i] = img2

        yield [X1, X2], Y


# Train the model
history = model.fit(
    generate_batch(pairs_train, labels_train, batch_size),
    steps_per_epoch=len(labels_train) // batch_size,
    epochs=epochs,
    validation_data=generate_batch(pairs_test, labels_test, batch_size),
    validation_steps=len(labels_test) // batch_size
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    generate_batch(pairs_test, labels_test, batch_size),
    steps=len(labels_test) // batch_size
)

print(f"Test Accuracy: {test_accuracy}")




Epoch 1/10


 45/168 [=======>......................] - ETA: 6:48 - loss: 0.0654 - accuracy: 0.9743

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
OSError: image file is truncated (23 bytes not processed)
Traceback (most recent call last):

  File "c:\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "c:\Python311\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "c:\Python311\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Python311\Lib\site-packages\keras\src\engine\data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\saish\AppData\Local\Temp\ipykernel_42104\761347596.py", line 110, in g

In [ ]:
'''
# Placeholder for iris recognition model
# Define iris recognition model (This is a placeholder, replace it with your actual iris recognition model)
iris_model = ... # Define your iris recognition model here

# Score level fusion
def score_level_fusion(face_scores, iris_scores):
    # Fusion strategy: For simplicity, let's take the average of scores
    fused_scores = (face_scores + iris_scores) / 2
    return fused_scores

# Predict labels for test images using the iris model and print filenames along with predicted labels
print("\nImage File Name | Predicted Label (Fused) | Match")
print("---------------------------------------------------")
for i in range(len(X_test)):
    # Get the filename of the current image
    filename = os.listdir(os.path.join(base_directory, test_dir))[i]
    # Extract the first 5 characters from the filename
    filename_prefix = filename[:5]
    # Predict the label for the current image using the iris model (This is a placeholder)
    iris_predicted_label = ... # Replace this with actual prediction using iris model
    # Fuse the scores from face and iris models
    fused_scores = score_level_fusion(predicted_label, iris_predicted_label)
    # Decide the final label based on fused scores
    final_predicted_label = np.argmax(fused_scores)
    # Check if the predicted label matches the first 5 characters of the filename
    match = "Yes" if str(final_predicted_label) == filename_prefix else "No"
    # Print the filename, predicted label, and whether there's a match
    print("{:<15} | {:<25} | {:<5}".format(filename, final_predicted_label, match))
'''